# Import & Setup

In [ ]:
import sys
sys.path.append(r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/My-Package")

import pandas as pd
import numpy as np
from myfunctions import clean_path
import h3


In [ ]:
cols_to_use = ['Accident - Sparse Filled',
 'Belts - Sparse Filled',
 'Personal Injury - Sparse Filled',
 'Property Damage - Sparse Filled',
 'Fatal - Sparse Filled',
 'Commercial License - Sparse Filled',
 'Alcohol - Sparse Filled',
 'Work Zone - Sparse Filled',
 'Contributed To Accident - Sparse Filled',
 'Citation - Sparse Filled',
 'Warning - Sparse Filled',
 'Male - Sparse Filled',
 'Female - Sparse Filled',
 'Probable Cause - Sparse Filled',
 'Arrest - Sparse Filled',
 'Agency',
 'SubAgency',
 'Description',
 'Location',
 'Latitude',
 'Longitude',
 'Stop ID']

In [ ]:
df = pd.read_csv(filepath_or_buffer=clean_path(r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/Policing Thesis/Modified Dataset - 2021 - One Row per Stop.csv"))

In [ ]:

df = pd.read_csv(filepath_or_buffer=clean_path(r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/Policing Thesis/Modified Dataset - 2021 - One Row per Stop.csv"),
                 index_col="Stop ID",
                 usecols=cols_to_use)

## Remove " - Sparse Filled" from columns names

In [ ]:
df.columns = [col.replace(" - Sparse Filled", "") for col in df]

# H3 Encoding

* Resolution options:
  * Official list w/ area https://h3geo.org/docs/core-library/restable/
  * Visualize at https://observablehq.com/@four43/h3-index-visualizer
* Looks like resolution=(7 or 8) is the way to go

In [ ]:
def create_h3_series(dataframe:pd.DataFrame, resolution:int) -> pd.Series:
    return h3.geo_to_h3(lat=dataframe['Longitude'], 
                        lng=dataframe['Latitude'],
                        resolution=resolution)

In [ ]:
df['H3 Encoding - Res=7'] = df.apply(create_h3_series, resolution=7, axis=1)
df['H3 Encoding - Res=8'] = df.apply(create_h3_series, resolution=8, axis=1)

# Aggregate

Get # Stops in a cell, # Accidents, # Fatalities, # 

In [ ]:
res7_df = df.groupby(by='H3 Encoding - Res=7').sum()
res8_df = df.groupby(by='H3 Encoding - Res=8').sum()

In [ ]:
res7_df['']

In [ ]:
def aggregate_by_hex(dataframe:pd.DataFrame, resolution:int):
    grouped = df.groupby(by=f'H3 Encoding - Res={resolution}')
    

# Plotly

In [ ]:
import plotly.figure_factory as ff
import plotly.express as px

fig = ff.create_hexbin_mapbox(
    data_frame=df, lat="Latitude", lon="Longitude",
    nx_hexagon=10, opacity=0.9, labels={"color": "Point Count"},
)
fig.update_layout(margin=dict(b=0, t=0, l=0, r=0))
fig.show()